In [1]:
import sys
import os
here = os.getcwd()
sys.path.append(os.path.join(here,"../"))

In [2]:
#!/usr/bin/env python
# coding: utf-8

import dcarte



import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix,precision_score,recall_score,accuracy_score,roc_auc_score
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree,export_text,export_graphviz

import ast
import math
import operator

op_map = {'>=':operator.ge,'>':operator.gt,'<=':operator.le,'<':operator.lt,'==':operator.eq}




import torch
import umap
import shap
import gc


In [3]:
from utils.test_utils import make_results_filenames

#from explainer.integrad import integrad
from explainer.FPGrowth_tree import *
from explainer.rule_pattern_miner import *
from explainer.explainer_utils import *
import explainer.RuleGrowth_tree as rgtree


In [4]:

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--dpath',default='../data', help='Path to data file')
parser.add_argument('--rpath',default='../experiments/results', help='Path to save results')
parser.add_argument('--model_path',default='./results/models', help='Path to load model')
parser.add_argument('--use_phys',action='store_true', help='Use physiology data or not')
parser.add_argument('--seed',default=0,type=int, help='Random seed')
parser.add_argument('--model_name',default='ncde',type=str, help='Model name')
parser.add_argument('--device',default='cpu',type=str, help='cpu or cuda')
parser.add_argument('--side_input',action='store_true', help='Use side input to final task')
parser.add_argument('--concat_z',action='store_true', help='Concat hidden states for the final task')
parser.add_argument('--time_intensity',action='store_true', help='Add time intensity')
parser.add_argument('--intensity',action='store_true', help='Add X intensity')
parser.add_argument('--time_len',default=12,type=int, help='Length of time indices')
parser.add_argument('--hidden_channels',default=4,type=int, help='Dimension of hidden states z')
parser.add_argument('--hidden_hidden_channels',default=32,type=int, help='Dimension of hidden units of f')
parser.add_argument('--num_hidden_layers',default=4,type=int, help='Number of hidden layers of f')
parser.add_argument('--interpolate',default='cubic_spline',type=str, help='Interpolation function name')
parser.add_argument('--batch_size',default=1024,type=int, help='Batch size')
parser.add_argument('--pos_weight',default=12,type=int, help='Weight of positive class')
parser.add_argument('--max_epochs',default=500,type=int, help='Maximum epochs')
parser.add_argument('--lr',default=0.0001,type=float, help='Raw learning rate')
parser.add_argument('--K',default=5,type=int, help='K-fold cross-validation')

_StoreAction(option_strings=['--K'], dest='K', nargs=None, const=None, default=5, type=<class 'int'>, choices=None, required=False, help='K-fold cross-validation', metavar=None)

In [5]:
args,_ = parser.parse_known_args()


pp = "/Users/chenyu/github/NDE-Models-DigitalHealth/"
args.dpath = pp+"data/processed/diabetes_prediction_dataset.csv"
print(args)

Namespace(dpath='/Users/chenyu/github/NDE-Models-DigitalHealth/data/processed/diabetes_prediction_dataset.csv', rpath='../experiments/results', model_path='./results/models', use_phys=False, seed=0, model_name='ncde', device='cpu', side_input=False, concat_z=False, time_intensity=False, intensity=False, time_len=12, hidden_channels=4, hidden_hidden_channels=32, num_hidden_layers=4, interpolate='cubic_spline', batch_size=1024, pos_weight=12, max_epochs=500, lr=0.0001, K=5)


In [6]:
name = os.path.join(args.rpath,'diabetes')
num_classes = 2
#output_channels = 1

In [7]:
df = pd.read_csv(args.dpath,header=1)
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [8]:
df.diabetes.sum(),df.shape

(8500, (100000, 9))

In [9]:
def encode_one_hot(df,col_name):
    df[col_name] = pd.Categorical(df[col_name])
    one_hot = pd.get_dummies(df[col_name])
    df = pd.concat([df, one_hot], axis=1)
    df = df.drop(col_name,axis=1)
    return df

In [10]:
def min_max_normalization(df,col_name):
    min_val = df[col_name].min()
    max_val = df[col_name].max()
    df[col_name] = (df[col_name]-min_val)/(max_val-min_val)
    return df

In [11]:
df = min_max_normalization(df,'age')
df = min_max_normalization(df,'bmi')
df = min_max_normalization(df,'HbA1c_level')
df = min_max_normalization(df,'blood_glucose_level')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,1.000000,0,1,never,0.177171,0.563636,0.272727,0
1,Female,0.674675,0,0,No Info,0.202031,0.563636,0.000000,0
2,Male,0.349349,0,0,never,0.202031,0.400000,0.354545,0
3,Female,0.449449,0,0,current,0.156863,0.272727,0.340909,0
4,Male,0.949950,1,1,current,0.118231,0.236364,0.340909,0


In [12]:
df = encode_one_hot(df,'gender')
df = encode_one_hot(df,'smoking_history')
df.head()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,Female,Male,Other,No Info,current,ever,former,never,not current
0,1.000000,0,1,0.177171,0.563636,0.272727,0,1,0,0,0,0,0,0,1,0
1,0.674675,0,0,0.202031,0.563636,0.000000,0,1,0,0,1,0,0,0,0,0
2,0.349349,0,0,0.202031,0.400000,0.354545,0,0,1,0,0,0,0,0,1,0
3,0.449449,0,0,0.156863,0.272727,0.340909,0,1,0,0,0,1,0,0,0,0
4,0.949950,1,1,0.118231,0.236364,0.340909,0,0,1,0,0,1,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop('diabetes',axis=1),df['diabetes'],test_size=0.3,random_state=42)

In [14]:
from imblearn.over_sampling import SMOTE
oversampler = SMOTE()
X_over, y_over = oversampler.fit_resample(X_train, y_train)

In [15]:
log_reg = LogisticRegression(max_iter=2000,multi_class='ovr')
log_reg.fit(X_over,y_over)
y_pred = log_reg.predict(X_test)


In [16]:
accuracy = log_reg.score(X_test,y_test)*100

confusion = confusion_matrix(y_test,y_pred)
precision = precision_score(y_test,y_pred,average=None)
recall = recall_score(y_test,y_pred,average=None)
accuracy,confusion,precision,recall

(88.71,
 array([[24366,  3087],
        [  300,  2247]]),
 array([0.98783751, 0.42125984]),
 array([0.88755327, 0.88221437]))

In [17]:
log_reg.coef_.shape,log_reg.intercept_

((1, 15), array([-13.17684315]))

In [18]:
Z = X_train.values * log_reg.coef_

In [19]:
np.abs(Z).mean()

0.7143407948523366

In [20]:
 log_reg.coef_

array([[ 4.05355194,  0.72462918,  0.65559644,  7.7464791 , 12.09342134,
         6.87918845,  0.33091601,  0.59998099, -0.93506938, -0.50103273,
         0.20635978,  0.11833448,  0.08991172, -0.0287554 ,  0.11100978]])

In [21]:
z_test = X_test.values * log_reg.coef_

In [22]:
z_test.sum(axis=1).min()

0.10555014959822184

In [23]:
itemsets = transform_intgrad_to_itemsets(Z,thd=0.7,K=1)

In [24]:
itemsets

{0: [[1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 6.0],
  [1.0, 4.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [4.0, 5.0, 6.0],
  [1.0, 4.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 2.0, 4.0, 5.0, 6.0],
  [4.0, 5.0, 6.0],
  [5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 6.0],
  [1.0, 4.0, 5.0],
  [1.0, 4.0, 5.0, 

In [25]:
y_z = Z.sum(axis=1)+log_reg.intercept_
Y = y_train.values

In [26]:
input_feature_names = X_train.columns

In [27]:


max_depth = 3 
snr_th=0.
prob_low_th=0.02
prob_high_th=0.2
min_samples_leaf=2000
min_support=2000

y = y_train.values
x = X_train.values
K = 1

#z_patterns = {}
z_rules = {}
#all_rules = []

for l in range(K):
    print('### latent state {} ###'.format(l))    

    itemsets_z = itemsets[int(l)]
    
    z_rules[l] = find_pattern_by_latent_state(x,y,y_z,itemsets_z,zw_pos=True,c=1,num_grids=20,omega=0.1,min_support_pos=2000,min_support_neg=2000,max_depth=max_depth)


### latent state 0 ###
min -12.894063569236238 max 13.622999880474458
p(y=1) 0.08504285714285714
thd_h 3.8535554516336745 thd_l -3.1246191403954544 pos True
[4.0] 65170.0
[4.0, 5.0] 60253.0
[4.0, 5.0, 1.0] 55666.0
[4.0, 5.0, 1.0, 6.0] 40310.0
[4.0, 5.0, 1.0, 6.0, 2.0] 3914.0
[4.0, 5.0, 6.0] 3159.0
[4.0, 1.0] 4521.0
[4.0, 1.0, 6.0] 3163.0
[5.0] 4429.0
[5.0, 6.0] 2773.0
feature comb [3 4 0]
build_rule_tree_one_side
init rule tree
num potential rules 3
add rule {'fid': 4, 'rule': (18.43088243554457, 0.5714285714285714, 1.0, 2736)}
num potential rules 0
add rule {'fid': 0, 'rule': (2.1453767599992415, 0.6122448979591836, 1.0, 28055)}
num potential rules 2
add rule {'fid': 4, 'rule': (9.06676488416806, 0.5714285714285714, 1.0, 2193)}
num potential rules 0
add rule {'fid': 3, 'rule': (1.6092028058825742, 0.21435002572457554, 0.9548319327731092, 10560)}
num potential rules 0
add rule {'fid': 3, 'rule': (2.100038103784151, 0.21435002572457554, 0.7015091750986109, 20704)}
num potential rules 1


In [28]:
z_rules[0]

{'pos': True,
 'thd_h': 3.8535554516336745,
 'thd_l': -3.1246191403954544,
 'p(z>=thd_h)': 0.040157142857142857,
 'p(z<=thd_l)': 0.4752142857142857,
 'rule_dict_higher_z': {(4,): {'rules': [(4, '>=', 0.5714285714285714)],
   'cond_prob_z': 0.7401315789473685,
   'cond_prob_y': 1.0,
   'ratio_y': 0.4596002015790358,
   'support': 2736},
  (0,
   4): {'rules': [(0, '>=', 0.6122448979591836),
    (4,
     '>=',
     0.5714285714285714)], 'cond_prob_z': 0.7811217510259918, 'cond_prob_y': 1.0, 'ratio_y': 0.3683856878884596, 'support': 2193},
  (0,
   3): {'rules': [(0, '>=', 0.6122448979591836),
    (3,
     '>=',
     0.21435002572457554)], 'cond_prob_z': 0.13863636363636364, 'cond_prob_y': 0.25729166666666664, 'ratio_y': 0.4564085335125147, 'support': 10560},
  (3,
   0): {'rules': [(3, '>=', 0.21435002572457554),
    (3, '<=', 0.7015091750986109),
    (0,
     '>=',
     0.673469387755102)], 'cond_prob_z': 0.15082715345122646, 'cond_prob_y': 0.2772390188248717, 'ratio_y': 0.4081975474550

In [29]:
sorted_rules_pos = sort_rules(z_rules,input_feature_names,pos=True)
sorted_rules_pos 

0 True
rule_dict_higher_z p(z>=thd_h)


[{'rules': [(4, 'HbA1c_level', '>=', 0.5714285714285714)],
  'zid': 0,
  'p(z>=thd_h)': 0.040157142857142857,
  'thd_h': 3.8535554516336745,
  'cond_prob_y': 1.0,
  'cond_prob_z': 0.7401315789473685,
  'support': 2736,
  'ratio_y': 0.4596002015790358},
 {'rules': [(0, 'age', '>=', 0.6122448979591836),
   (4, 'HbA1c_level', '>=', 0.5714285714285714)],
  'zid': 0,
  'p(z>=thd_h)': 0.040157142857142857,
  'thd_h': 3.8535554516336745,
  'cond_prob_y': 1.0,
  'cond_prob_z': 0.7811217510259918,
  'support': 2193,
  'ratio_y': 0.3683856878884596},
 {'rules': [(3, 'bmi', '>=', 0.21435002572457554),
   (3, 'bmi', '<=', 0.7015091750986109),
   (0, 'age', '>=', 0.673469387755102)],
  'zid': 0,
  'p(z>=thd_h)': 0.040157142857142857,
  'thd_h': 3.8535554516336745,
  'cond_prob_y': 0.2772390188248717,
  'cond_prob_z': 0.15082715345122646,
  'support': 8765,
  'ratio_y': 0.40819754745506465}]

In [30]:
sorted_rules_neg = sort_rules(z_rules,input_feature_names,pos=False)
sorted_rules_neg

0 True
rule_dict_lower_z p(z<=thd_l)


[{'rules': [(4, 'HbA1c_level', '<=', 0.3877551020408163),
   (0, 'age', '<=', 0.5918367346938775),
   (3, 'bmi', '<=', 0.21435002572457554)],
  'zid': 0,
  'p(z<=thd_l)': 0.4752142857142857,
  'thd_l': -3.1246191403954544,
  'cond_prob_y': 1.0,
  'cond_prob_z': 0.9849975802548798,
  'support': 12398,
  'ratio_y': 0.1935765921901104},
 {'rules': [(4, 'HbA1c_level', '<=', 0.3877551020408163),
   (3, 'bmi', '<=', 0.21435002572457554),
   (0, 'age', '<=', 0.6122448979591836)],
  'zid': 0,
  'p(z<=thd_l)': 0.4752142857142857,
  'thd_l': -3.1246191403954544,
  'cond_prob_y': 1.0,
  'cond_prob_z': 0.9820835585759519,
  'support': 12949,
  'ratio_y': 0.2021796493200306},
 {'rules': [(3, 'bmi', '<=', 0.17537729377465272),
   (4, 'HbA1c_level', '<=', 0.4081632653061224),
   (0, 'age', '<=', 0.4693877551020408)],
  'zid': 0,
  'p(z<=thd_l)': 0.4752142857142857,
  'thd_l': -3.1246191403954544,
  'cond_prob_y': 0.999559277214632,
  'cond_prob_z': 0.9836932569413839,
  'support': 6807,
  'ratio_y': 

In [31]:
X_train

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,Female,Male,Other,No Info,current,ever,former,never,not current
76513,0.612112,0,0,0.202031,0.272727,0.340909,0,1,0,0,0,1,0,0,0
60406,0.799800,0,0,0.202031,0.000000,0.295455,0,1,0,0,0,0,0,1,0
27322,0.299299,0,0,0.202031,0.000000,0.227273,1,0,0,0,1,0,0,0,0
53699,0.687187,0,0,0.202031,0.545455,0.359091,1,0,0,0,1,0,0,0,0
65412,0.174174,0,0,0.128035,0.490909,0.022727,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,0.612112,0,0,0.268091,0.400000,0.000000,0,1,0,0,0,0,0,1,0
54886,0.186687,0,0,0.211134,0.272727,0.359091,1,0,0,0,0,0,0,1,0
76820,0.524525,0,0,0.188259,0.418182,0.022727,0,1,0,0,0,0,0,1,0
860,0.461962,0,0,0.174486,0.490909,0.354545,1,0,0,0,0,0,0,1,0


In [32]:
tid = 6
xi = X_train.values[tid]
xi

array([0.54954955, 0.        , 0.        , 0.18475724, 0.45454545,
       0.04545455, 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ])

In [33]:
zi = y_z[tid]
zi,y[tid]

(-3.4061367523481305, 0)

In [34]:
if y[tid] == 1:
    match = match_sample_rules(xi,sorted_rules_pos)
else:
    match = match_sample_rules(xi,sorted_rules_neg)

In [35]:
if match is None:
    x_z_rules = find_pattern_by_sample_latent_state(xi,zi,x,y,y_z,itemsets[0],zw_pos=True,c=1,num_grids=20,omega=0.1,min_support_pos=2000,min_support_neg=2000,max_depth=5)

min -12.894063569236238 max 13.622999880474458
p(y=1) 0.08504285714285714
raw thd 3.8535554516336745 -3.1246191403954544
thd -3.1246191403954544 pos True
[4.0] 65170.0
[4.0, 5.0] 60253.0
[4.0, 5.0, 1.0] 55666.0
[4.0, 5.0, 1.0, 6.0] 40310.0
[4.0, 5.0, 1.0, 6.0, 2.0] 3914.0
[4.0, 5.0, 6.0] 3159.0
[4.0, 1.0] 4521.0
[4.0, 1.0, 6.0] 3163.0
[5.0] 4429.0
[5.0, 6.0] 2773.0
feature comb [3 4 0 5 1]
build_rule_tree_one_side
init rule tree
num potential rules 2
add rule {'fid': 5, 'rule': (1.3197736709187076, 0.0, 0.26530612244897955, 30513)}
num potential rules 1
add rule {'fid': 1, 'rule': (1.0354678276562035, 0.0, 0.9795918367346939, 28527)}
num potential rules 0
add rule {'fid': 1, 'rule': (1.0518804106800068, 0.0, 0.9795918367346939, 64687)}
num potential rules 1
add rule {'fid': 5, 'rule': (1.29918112567625, 0.0, 0.26530612244897955, 28527)}
num potential rules 0
remove duplicate rules (5, 1) [(5, '>=', 0.0), (5, '<=', 0.26530612244897955), (1, '>=', 0.0), (1, '<=', 0.9795918367346939)] (1,

In [36]:
x_z_rules

{'pos': True,
 'thd_l': -3.1246191403954544,
 'p(z<=thd_l)': 0.4752142857142857,
 'rule_dict_lower_z': {(5,
   1): {'rules': [(5, '<=', 0.26530612244897955),
    (1,
     '<=',
     0.9795918367346939)], 'cond_prob_z': 0.6494198478634277, 'cond_prob_y': 0.975602061205174, 'ratio_y': 0.4345402594969319, 'support': 28527}}}

In [37]:
sort_rules({0:x_z_rules},input_feature_names,pos=False)

0 True
rule_dict_lower_z p(z<=thd_l)


[{'rules': [(5, 'blood_glucose_level', '<=', 0.26530612244897955),
   (1, 'hypertension', '<=', 0.9795918367346939)],
  'zid': 0,
  'p(z<=thd_l)': 0.4752142857142857,
  'thd_l': -3.1246191403954544,
  'cond_prob_y': 0.975602061205174,
  'cond_prob_z': 0.6494198478634277,
  'support': 28527,
  'ratio_y': 0.4345402594969319}]